# Introduction

Bits are the world's simplest alphabet, allowing any piece of information to be represented using the characters 0 and 1. Qubits are an extension of the bit to quantum mechanics, allowing us to encode arbitrary quantum information with just the states $|0\rangle$ and $|1\rangle$. But what if intead of two characters, the alphabet were infinite?

A quantum harmonic oscillator is an object with an infinite alphabet of states. By "state," we mean the different possible energy levels of the oscillator. One of the consequences of quantum mechanics is that energy can only be added to the oscillator in discrete packets. These packets are called photons. The amount of energy in the oscillator increases in proportion to the number of photons in the oscillator, so the different states of the oscillator are labeled by the number of photons corresponding to that state, i.e. $|0\rangle, |1\rangle, |2\rangle, |3\rangle,...$, all the way to infinity.

insert diagram here

## Dirac Notation
You may be familiar with representing quantum states using matrices with complex coefficients. There is another convenient way to represent quantum states called Dirac notation. Dirac notation denotes a state by placing a label for a state between $|$ and $\rangle$. Mutually exclusive states, such as states with different enegies or a different number of photons, are assigned to orthogonal vectors. This allows us to easily go back and forth between matrix notation and Dirac notation:
$$
\alpha |0\rangle + \beta |1 \rangle \ \dot{=} \ \alpha \begin{pmatrix} 1 \\ 0 \end{pmatrix} + \beta \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \begin{pmatrix} \alpha \\ \beta \end{pmatrix}
$$
The funny $\dot =$ sign is used to indicate that the choice of orthogonal vectors to assign to each state is arbitrary, so we could easily have made a different choice to assign $|0 \rangle$ to $\begin{pmatrix} 0 \\ 1\end{pmatrix}$ and $|1\rangle$ to $\begin{pmatrix} 1 \\ 0 \end{pmatrix}$.

One obvious advantage of Dirac notation is that it removes the ambiguity representing states as matrices. Another advantage is particularly clear with a quantum harmonic oscillator. With an infinite number of mutually exclusive states, the vectors would be infinitely long!

## States

### Superposition
Qubits have the familiar basis states $|0\rangle$ and $|1\rangle$. The "state space" of the qubit is a vector space with coefficients that are complex numbers. These basis states act like vectors in the state space. Like vectors, they can be multiplied by any complex number and added to get another state. The fact that these states form a *basis* implies two properties: 
1) they are mutually exclusive
2) if $|\psi\rangle$ is any valid state, then it is possible to find complex numbers $\alpha$ and $\beta$ such that
$$
|\psi \rangle = \alpha |0\rangle + \beta |1\rangle
$$

These assertions hold for a quantum harmonic oscillator as well, except now we have an infinite number of states $|0\rangle, |1\rangle, |2\rangle,...$ to work with. Each of these states corresponds to a different number of photons in the oscillator, so they are mutually exclusive. Furthermore, for any state of the oscillator $|\psi\rangle$, it is possible to find a series of complex numbers $\alpha_0, \alpha_1, \alpha_2,...$ such that
$$
|\psi\rangle = \sum_{n=0}^\infty \alpha_n |n\rangle
$$

### Borne Rule
An important postulate of quantum mechanics is the *Borne Rule*. This postulate is the foundation for interpreting the physical meaning of superpositions and predicting the outcome of measurements. Measurement is one of the most crucial parts of a quantum algorithm--it's how we obtain the results! Measuring a classical bit is easy; The bit is either in the state $0$ or $1$, so just look at it and see what state it is in. Quantum mechanics complicates this simple picture by allowing superpositions like the one above:
$$
|\psi \rangle = \alpha |0 \rangle + \beta | 1 \rangle
$$
What would the outcome of a measurement on this state be? The Borne rule gives an answer. Take the qubit state $|0\rangle$ without loss of generality. The Borne Rule says that the probability of measuring $|\psi\rangle$ in the state $|0\rangle$ is 
$$
p_\psi(0) = \langle \psi | 0 \rangle \langle 0 | \psi \rangle = |\langle 0 | \psi \rangle |^2
$$

Where the notation $\langle \psi |$ is called a "bra", and is like a row-vector, or a linear map that sends kets to complex numbers. Multiplying a bra and a ket is a type of operation called an inner product. The Borne Rule tells us something that we assumed when constructing matrix representations: the inner product of two mutually exclusive states should be zero. 

Why should this be the case? Take for example the states $|0\rangle$ and $|1\rangle$. These states are mutually exclusive, and so the probability of measuring $|0\rangle$ in the state $|1\rangle$ should be zero. By the Borne rule, $p_0(1) = |\langle 1 | 0 \rangle|^2 = 0$. This shows that mutually exclusive states are orthogonal, or in other words their inner product is zero.

Another important mechanism in quantum mechanics is state collapse. In quantum mechanics, you don't see what you get; You get what you see. This summarizes the phenomenon of state collapse. When an experimenter measures a set of mutually exclusive (orthogonal) states, the measurement, no matter how delicate, causes an irresversible collapse of the system into the state consistent with the result of the measurement.

### Normalization
We said previously that state space is like a vector space in that multiplying two states by complex numbers and adding them should produce another vector in state space, but this is not entirely correct. The Borne rule places another restriction on the vectors in state space: normalization. We define the norm of a state $|\psi\rangle$ to be the square root of the inner product of a state with itself:
$$
\Vert \psi \Vert \equiv \sqrt{\langle \psi | \psi \rangle}
$$
The probability of measuring the state $|\psi\rangle$ in the state $|\psi\rangle$, if thre is any sense in this, should be one. By the Borne Rule, this implies that
$$
p_\psi(\psi) = |\langle \psi | \psi \rangle|^2 = 1
$$
Another way to say this is $\Vert \psi \Vert = 1$. This is called normalization, and the consequence is that all valid quantum states must be normalized.

### Generalizing to Bosonic Modes:
All of the above assertions hold for quantum harmonic oscillators:
1. Orthogonality: $|n\rangle$ and $|m\rangle$ are mutually exclusive if $m\neq n$, so $\langle m | n \rangle = 0$. 
2. Normalization: All states are normalized, so $\langle n | n \rangle = 1$ for any $n$.
These last two properties can be summarized as
$$
\langle m | n \rangle = \delta_{mn}
$$
Where the symbol $\delta_{mn}$ is known as a Kronecker delta.

3. The Borne Rule: If $|\psi\rangle = \sum_{n=0}^\infty \alpha_n |n\rangle$ is a general state, then
$$
p_\psi(m) = |\langle m | \psi\rangle|^2 = \left|\langle m | \sum_{n=0}^\infty \alpha_n|n\rangle \right|^2
=\left|\sum_{n=0}^\infty \alpha_n\langle m|n\rangle \right|^2 = |\alpha_m|^2
$$
